In [2]:

def resolve_proxy():
    import os
    os.environ['HTTP_PROXY']="http://10.8.0.1:8080"
    os.environ['HTTPS_PROXY']="http://10.8.0.1:8080"
    os.environ['http_proxy'] = "http://10.8.0.1:8080" 
    os.environ['https_proxy'] = "http://10.8.0.1:8080" 
resolve_proxy()

In [3]:
# https://dipy.org/documentation/1.6.0./examples_built/tracking_introduction_eudx/#example-tracking-introduction-eudx
# Enables/disables interactive visualization
interactive = False

from dipy.core.gradients import gradient_table
from dipy.data import get_fnames
from dipy.io.gradients import read_bvals_bvecs
from dipy.io.image import load_nifti, load_nifti_data

hardi_fname, hardi_bval_fname, hardi_bvec_fname = get_fnames('stanford_hardi')
# importing stanford hardi data with get_fnames
label_fname = get_fnames('stanford_labels')

data, affine, hardi_img = load_nifti(hardi_fname, return_img=True)
labels = load_nifti_data(label_fname) # read labels of the NIFTI data

bvals, bvecs = read_bvals_bvecs(hardi_bval_fname, hardi_bvec_fname)
gtab = gradient_table(bvals, bvecs)

/home/ang/anaconda3/envs/SupWMA/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 1/1 [00:00<00:00, 785.01 MB/s]


In [6]:
hardi_fname, hardi_bval_fname, hardi_bvec_fname, label_fname

('/home/ang/.dipy/stanford_hardi/HARDI150.nii.gz',
 '/home/ang/.dipy/stanford_hardi/HARDI150.bval',
 '/home/ang/.dipy/stanford_hardi/HARDI150.bvec',
 '/home/ang/.dipy/stanford_hardi/aparc-reduced.nii.gz')